In [ ]:
from IPython.display import IFrame, Image

## Мотивация создания Apache Spark

### Рассмотрим два примера приложений:
- Обучить модель на больших данных (читай итеративный алгоритм над фиксированным датасетом)
- Провести ad-hoc анализ данных из двух таблиц (читай несколько интерактивных запросов с джойнами)

## Основные недостатки классического MapReduce
- Быстроумирающие контейнеры
- Постоянное чтение/запись во внешнее хранилище
- Сложный API
- Ограниченное число источников/приемников данных
- MapReduce - это только вычислительный фреймворк

## Apache Spark - это *быстрая* распределенная вычислительная платформа *общего назначения*
1. **Быстрая** - это в памяти и с ленивыми вычислениями
2. **Общего назначения** - значит на ней можно реализовать любые вычисления (батчевые, интерактивные, итеративные, в режиме реального времени)

<img src="pics/spark_stack.png" width=1000/>

## Множество источников данных

<img src="pics/spark_data_sources.jpg" width=1000/>

## Архитектура Apache Spark

<img src="pics/cluster-overview.png" width=800/>

## SparkContext (sc) - это основной управляющий объект.

In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

## Для получения всех установленных опций конфигурации можно использовать `sc.getConf()`

In [2]:
sc.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.eventLog.dir', 'hdfs://brain-master.bigdatateam.org/spark/logs'),
 ('spark.executor.instances', '2'),
 ('spark.dynamicAllocation.maxExecutors', '12'),
 ('spark.ui.proxyBase', '/proxy/application_1602972439113_9908'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'brain-master.bigdatateam.org'),
 ('spark.driver.host', 'brain-client.bigdatateam.org'),
 ('spark.history.ui.port', '18080'),
 ('spark.shuffle.service.enabled', 'true'),
 ('spark.history.fs.logDirectory',
  'hdfs://brain-master.bigdatateam.org/spark/logs'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.appUIAddress', 'http://brain-client.bigdatateam.org:4040'),
 ('spark.yarn.historyServer.address

# RDD (Resilient Distributed Dataset)

## Существует два способа создать RDD
- распределить коллекцию объектов с драйвера
- загрузить внешний датасет

## 1. Распределить коллекцию с драйвера

In [3]:
import numpy as np
vocabulary = ("Apache", "Spark", "Hadoop")
numbers = np.random.randint(10, size=10000)
words = np.random.choice(vocabulary, size=10000)
collection = zip(numbers, words)

In [4]:
rdd = sc.parallelize(collection)

In [5]:
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [6]:
rdd.count()

10000

In [7]:
rdd.take(10)

[(9, 'Spark'),
 (9, 'Hadoop'),
 (3, 'Spark'),
 (7, 'Hadoop'),
 (5, 'Spark'),
 (4, 'Apache'),
 (8, 'Apache'),
 (6, 'Spark'),
 (6, 'Apache'),
 (9, 'Spark')]

## 2. Загрузить внешний датасет (датасет загружается из HDFS)

In [8]:
!hdfs dfs -ls /data/spark/lecture05/ips.txt

-rw-r--r--   3 hdfs hdfs      19462 2020-08-11 20:22 /data/spark/lecture05/ips.txt


In [13]:
rdd2 = sc.textFile("/data/spark/lecture05/ips.txt")

In [14]:
rdd2.take(10)

['192.168.0.1\tCHINA',
 '192.168.0.2\tCHINA',
 '192.168.0.3\tCHINA',
 '192.168.0.4\tCHINA',
 '192.168.0.5\tCHINA',
 '192.168.0.6\tCHINA',
 '192.168.0.7\tCHINA',
 '192.168.0.8\tCHINA',
 '192.168.0.9\tCHINA',
 '192.168.0.10\tCHINA']

In [12]:
rdd2.count()

1000

## RDD API состоит из операции двух типов:
- action
- transformation

### Трансформация преобразовывает RDD в другой RDD и не приводит к вычислениям

In [17]:
rdd = sc.parallelize(range(100))

In [16]:
rdd

PythonRDD[13] at RDD at PythonRDD.scala:53

### Action заставляет Spark произвести вычисления и вернуть результат либо на драйвер, либо во внешнее хранилище

In [18]:
rdd.count()

100

In [19]:
rdd.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Трансформации можно применять одну за другой, никаких вычислений не будет сделано, пока не будет вызван action

In [ ]:
rdd

In [20]:
rdd2 = rdd.filter(lambda x: x % 2)
rdd2

PythonRDD[17] at RDD at PythonRDD.scala:53

In [21]:
rdd3 = rdd2.map(lambda x: x * 2)
rdd3

PythonRDD[18] at RDD at PythonRDD.scala:53

In [22]:
rdd3.collect()

[2,
 6,
 10,
 14,
 18,
 22,
 26,
 30,
 34,
 38,
 42,
 46,
 50,
 54,
 58,
 62,
 66,
 70,
 74,
 78,
 82,
 86,
 90,
 94,
 98,
 102,
 106,
 110,
 114,
 118,
 122,
 126,
 130,
 134,
 138,
 142,
 146,
 150,
 154,
 158,
 162,
 166,
 170,
 174,
 178,
 182,
 186,
 190,
 194,
 198]

## Как получить данные на драйвер и не выстрелить себе в ногу?

### `take()` пытается минимизировать число обращений к партициям, поэтому может возвращать смещенные результаты

In [23]:
rdd.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Будьте аккуратны с `collect()`, потому что он загружает все данные из RDD на драйвер. Это может легко привести к `Out of Memory exception`

In [ ]:
rdd.collect()[:20]

### Если нужно получить небольшое число записей на драйвер и, при этом, сохранить распределение, то лучше сделать выборку

In [24]:
rdd.takeSample(withReplacement=False, num=20, seed=5757)

[36, 75, 57, 18, 19, 96, 11, 55, 39, 40, 79, 26, 15, 13, 47, 7, 66, 86, 51, 12]

## Познакомимся с данными. Будем работать с двумя таблицами

![](pics/data_table1.png)

![](pics/data_table2.png)

### Примеры трансформаций

In [25]:
rdd = sc.textFile("/data/spark/lecture05/ips.txt")

In [26]:
rdd.take(5)

['192.168.0.1\tCHINA',
 '192.168.0.2\tCHINA',
 '192.168.0.3\tCHINA',
 '192.168.0.4\tCHINA',
 '192.168.0.5\tCHINA']

In [27]:
ips = rdd.map(lambda x: x.split("\t"))

In [28]:
ips.take(5)

[['192.168.0.1', 'CHINA'],
 ['192.168.0.2', 'CHINA'],
 ['192.168.0.3', 'CHINA'],
 ['192.168.0.4', 'CHINA'],
 ['192.168.0.5', 'CHINA']]

In [ ]:
ips_filtered = ips.filter(lambda x: x[1] != "CHINA")

In [ ]:
ips_filtered.take(5)

In [42]:
raw_logs = sc.textFile("/data/spark/lecture05/log.txt")

In [ ]:
raw_logs.take(5)

In [48]:
logs = raw_logs.map(lambda x: x.split("\t"))

In [ ]:
logs.take(5)

In [ ]:
logs.flatMap(lambda x: x[2].split()).take(20)

In [ ]:
words = logs.flatMap(lambda x: x[2].split())

In [ ]:
words.take(10)

In [ ]:
words.map(lambda word: (word, 1)).countByKey()

## Зачем нужны отдельные трансформации и отдельные action?

![](pics/dag1.png)

![](pics/dag2.png)

### Последовательность трансформаций определяет граф вычислений (DAG - direct acyclic graph). В нем есть партиции и зависимости между партициями. Таким образом Spark имеет всю необходимую информацию для вычилсения графа в любой точке и возможных оптимизаций

![](pics/dag3.png)

### Трансформации бывают *узкими*

![](pics/narrow_transformation.png)

### И *широкими*

![](pics/wide_transformation.png)

### Широкие трансформации разделяют джоб на стейджи. Между стейджами происходит shuffle данных, которого надо избегать

## Персистентность и кэширование

### RDD вычисляются лениво, когда вызывается action. Часто мы хотим вызвать несколько actions для одного и тоге же RDD. Если мы просто сделаем это, то граф будет полностью перевычисляться каждый раз.

In [29]:
ips.count()

1000

In [30]:
ips.top(10)

[['192.168.3.99', 'USA'],
 ['192.168.3.98', 'USA'],
 ['192.168.3.97', 'USA'],
 ['192.168.3.96', 'USA'],
 ['192.168.3.95', 'USA'],
 ['192.168.3.94', 'USA'],
 ['192.168.3.93', 'USA'],
 ['192.168.3.92', 'USA'],
 ['192.168.3.91', 'USA'],
 ['192.168.3.90', 'USA']]

### Чтобы этого избежать, мы можем закэшировать RDD в памяти. Кэширование произойдет при вызове первого action.

In [31]:
ips_cached = ips.cache()

In [32]:
ips_cached

PythonRDD[28] at RDD at PythonRDD.scala:53

In [33]:
ips_cached.count()

1000

In [34]:
ips_cached.top(20)

[['192.168.3.99', 'USA'],
 ['192.168.3.98', 'USA'],
 ['192.168.3.97', 'USA'],
 ['192.168.3.96', 'USA'],
 ['192.168.3.95', 'USA'],
 ['192.168.3.94', 'USA'],
 ['192.168.3.93', 'USA'],
 ['192.168.3.92', 'USA'],
 ['192.168.3.91', 'USA'],
 ['192.168.3.90', 'USA'],
 ['192.168.3.9', 'USA'],
 ['192.168.3.89', 'USA'],
 ['192.168.3.88', 'USA'],
 ['192.168.3.87', 'USA'],
 ['192.168.3.86', 'USA'],
 ['192.168.3.85', 'USA'],
 ['192.168.3.84', 'USA'],
 ['192.168.3.83', 'USA'],
 ['192.168.3.82', 'USA'],
 ['192.168.3.81', 'USA']]

In [35]:
!hdfs dfs -du -s -h /data/spark/lecture05/ips.txt

19.0 K  /data/spark/lecture05/ips.txt


In [ ]:
ips_cached

### `cache()` сохраняет RDD в памяти. Для большего контроля можно использовать `persist(storage_level)`:
+ MEMORY_ONLY
+ MEMORY_AND_DISK
+ DISK_ONLY
+ MEMORY_ONLY_2
+ MEMORY_AND_DISK_2

### Все сохраненные RDD можно увидеть во вкладке "Storage" Spark UI
### Или более программатичным способом

In [36]:
from pyspark import StorageLevel

In [ ]:
StorageLevel(False, True, False, False, 1)

In [37]:
ips.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [38]:
ips.unpersist()

PythonRDD[28] at RDD at PythonRDD.scala:53

In [39]:
ips.persist(StorageLevel.DISK_ONLY_2)

PythonRDD[28] at RDD at PythonRDD.scala:53

In [40]:
ips.getStorageLevel()

StorageLevel(True, False, False, False, 2)

In [41]:
ips.count()

1000

## PairRDD (ключ-значение)

### PairRDD - это RDD для работы с парами ключ-значение. Spark предполагает, что PairRDD содержить в себе объекты, состящие ровно из двух элементов! PairRDD предоставляют методы группировки, аггрегации и объединения (join) двух RDD

### Пусть есть задача подсчитать распределение кодов ERROR и WARNING в лог-файле

In [43]:
raw_logs.take(5)

['192.168.0.10\tERROR\tWhen production fails in dipsair, whom you gonna call?',
 '192.168.0.39\tINFO\tJust an info message passing by',
 '192.168.0.35\tINFO\tJust an info message passing by',
 '192.168.0.19\tINFO\tJust an info message passing by',
 '192.168.0.23\tERROR\tWhen production fails in dipsair, whom you gonna call?']

In [44]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .groupByKey()
         .map(lambda x: (x[0], len(x[1])))
         .collect())

[('ERROR', 49948), ('WARNING', 250004)]

### Или немного проще

In [45]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))
         .countByKey()
         .items())

dict_items([('ERROR', 49948), ('WARNING', 250004)])

### Стоит заметить, что `groupByKey()` предполагает перемещение всех записей с одним ключом на один экзекьютор. В случае очень скоршенных распределений это может привести к падению экзекьютора с OOM. Поэтому всегда при группировках стоит подумать об использовании `reduceByKey()`.

In [46]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .reduceByKey(lambda x, y: x + y)
         .collect())

[('ERROR', 49948), ('WARNING', 250004)]

## Join

### Два PairRDD можно объединить по ключу
### Поддерживаются inner join, left outer join, right outer join и full outer join

In [49]:
logs.take(5)

[['192.168.0.10',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?'],
 ['192.168.0.39', 'INFO', 'Just an info message passing by'],
 ['192.168.0.35', 'INFO', 'Just an info message passing by'],
 ['192.168.0.19', 'INFO', 'Just an info message passing by'],
 ['192.168.0.23',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?']]

In [50]:
ips.take(5)

[['192.168.0.1', 'CHINA'],
 ['192.168.0.2', 'CHINA'],
 ['192.168.0.3', 'CHINA'],
 ['192.168.0.4', 'CHINA'],
 ['192.168.0.5', 'CHINA']]

In [51]:
logs.join(ips).take(5)

[('192.168.0.55', ('INFO', 'CHINA')),
 ('192.168.0.55', ('INFO', 'CHINA')),
 ('192.168.0.55', ('INFO', 'CHINA')),
 ('192.168.0.55', ('INFO', 'CHINA')),
 ('192.168.0.55', ('INFO', 'CHINA'))]

![](pics/Jackie-Chan-WTF.jpg)

### Не стоит забывать, что Spark предполагает, что PairRDD состоит ровно! из двух элементов, поэтому все остальные элементы просто отбрасываются!

In [52]:
def split_logs(line):
    split = line.split("\t")
    return split[0], split[1:]

In [53]:
logs_cached = raw_logs.map(split_logs).cache()

In [54]:
logs_cached.take(5)

[('192.168.0.10',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?']),
 ('192.168.0.39', ['INFO', 'Just an info message passing by']),
 ('192.168.0.35', ['INFO', 'Just an info message passing by']),
 ('192.168.0.19', ['INFO', 'Just an info message passing by']),
 ('192.168.0.23',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?'])]

In [55]:
logs_cached.join(ips).take(5)

[('192.168.0.1', (['INFO', 'Just an info message passing by'], 'CHINA')),
 ('192.168.0.1', (['INFO', 'Just an info message passing by'], 'CHINA')),
 ('192.168.0.1', (['WARNING', 'Something bad could happen'], 'CHINA')),
 ('192.168.0.1', (['WARNING', 'Something bad could happen'], 'CHINA')),
 ('192.168.0.1', (['INFO', 'Just an info message passing by'], 'CHINA'))]

## Управление параллелизмом.

### Вспомним, что атомарным уровнем параллелизма в Spark является партиция. Об этом всегда стоит помнить, когда есть проблемы с производительностью приложения

In [56]:
logs.getNumPartitions()

4

### Метод `repartition()` может быть использован для изменения числа партиций.

In [57]:
logs = logs.repartition(8)

In [58]:
logs.getNumPartitions()

8

### `repartition()` всегда приводит к равномерному перераспределению данных, что ведет к shuffle. Если Вы уменьшаете число партиций, то стоит использовать `coalesce()`, который может избежать shuffle

In [59]:
logs = logs.coalesce(10)

In [60]:
logs.getNumPartitions()

8

In [61]:
logs = logs.coalesce(4)

In [62]:
logs.getNumPartitions()

4

### Узнать дефолтный уровень параллелизма можно из конфига. По-умолчанию, при работе с YARN, использукется общее число ядер, выделенных этому SparkContext на всех экзекьюторах, либо 2. Что больше.

In [63]:
sc.getConf().get("spark.default.parallelism")

In [64]:
sc.parallelize(range(100)).getNumPartitions()

2

## Broadcast

### Broadcast-объект - это неизменяемый объект, которая разделяется между всеми экзекьюторами
### Дистрибуция broadcast-объекта производится быстро и эффективно p2p-протоколом

### Реализуем map-side join с помощью broadcast-объекта

In [ ]:
ips_local = dict(ips.collect())

In [ ]:
ips_local['192.168.0.10']

In [ ]:
ips_broadcasted = sc.broadcast(ips_local)

In [ ]:
ips_broadcasted

In [ ]:
ips_broadcasted.value['192.168.0.10']

In [ ]:
logs_cached.take(5)

In [ ]:
def resolve_ip(row):
    return ips_broadcasted.value[row[0]], row[1:] ## row[0] is the IP address

In [ ]:
logs_cached.map(resolve_ip).take(10)

## Не забудьте погасить SparkContext!

In [ ]:
sc.stop()